In [12]:
import matplotlib
from matplotlib import pyplot as plt
from numpy import *
import numpy as np
import random
import scipy.io
import sklearn.utils

f1 = scipy.io.loadmat('train.mat')
%matplotlib inline

In [370]:
#Input Layer 28x28x1
#Convolution Layer
#Output Layer 10x1
#10 filters, 3x3x1


def reshape_x(x): #add bias row to NN input
    X = np.reshape(x,(len(x),len(x.T)))
    X = X/np.max(X)
    return X

def reshape_y(y,outputSize): #change y from a single value to an array of 10 values corresponding to output 
    tempy = np.zeros((len(y),outputSize))
    for i,r in enumerate(y):
        tempy[i][r[0]] = 1
    return tempy


class NeuralNet(): #single convNet with a single fully connected neural layer, pool layer?
    def __init__(self,output_Size,hidden_Size,filter_Size,filter_Count): #add bias later
        self.stride = 1
        self.padding = 0
        self.filter_Count = filter_Count

        filter_Weights = np.random.normal(0,0.01,filter_Count*filter_Size**2)
        self.filter_Weights = np.reshape(filter_Weights,(filter_Size**2,filter_Count))

        V = np.random.normal(0,0.01,hidden_Size*(filter_Count))
        self.V = np.reshape(V,(hidden_Size,filter_Count))

        W = np.random.normal(0,0.01,output_Size*(hidden_Size))
        self.W = np.reshape(W,(output_Size,hidden_Size))

        
        
    def forward(self, inputs):
        self.conv_Vals = np.dot(inputs.T, self.filter_Weights)
        self.activation_conv = self.activation_fnc(self.conv_Vals)
        self.hidden_Vals = np.dot(self.V,self.activation_conv.T) #transpose?
        self.activation_H = np.array([self.activation_fnc(np.sum(self.hidden_Vals,axis=1))]).T
        self.pre_guess = np.dot(self.W,self.activation_H)

        self.guess = self.activation_fnc(self.pre_guess)
        
    def meanSquared(self,inputs,outputs):
        self.forward(inputs)
        return sum((outputs-self.guess)**2)/2
    
    
    def meanSquaredBack(self,inputs,outputs): #might need to derive better
        self.forward(inputs)
#         outputs = np.repeat(outputs,self.filter_Count,axis=0).reshape(10,self.filter_Count) #change 10 to var
        back1 = (-outputs+self.guess)*self.activation_fnc(self.pre_guess,True)
        dW = np.dot(back1,self.activation_H.T)
        back2 = np.dot(self.W.T,back1)*self.activation_fnc(self.hidden_Vals,True)
        dV = np.dot(back2,self.activation_conv)
        back3 = np.dot(self.V.T,back2)*self.activation_fnc(self.conv_Vals,True).T
        dF = np.dot(back3,inputs.T).T
        return dF, dV, dW
        
        
    def update_Weights(self, updates, step_size):
        self.filter_Weights -= updates[0]*step_size
        self.V -= updates[1]*step_size
        self.W -= updates[2]*step_size
        
        
    def activation_fnc(self, z, deriv=False): #currently sigmoid
        a = None
        if deriv:
            a = (1-(1+np.exp(-z))**-1)/(1+np.exp(-z))
        else:
            a = (1+np.exp(-z))**-1
        nans = isnan(a)
        a[nans] = 0
        return a
    
    
    def train(self, inputs, outputs, iters, feat_size):
        e = np.exp(-5)
            
        self.inputs = inputs
        
        outputs = reshape_y(y,10)
        self.outputs = outputs
        
        for i in range(iters):
            r = random.randint(0,len(X)-1)
#             dp = im2col(inputs[r],feat_size,feat_size) #change to vars
#             dp = reshape_x(dp)
            dp = inputs[r]
            dpl = np.array([outputs[r]]).T
            deriv = self.meanSquaredBack(dp,dpl)
            if i % 1000 == 0:
                print(self.meanSquared(dp,dpl))
            self.update_Weights(deriv,e)
    
    
    def predict(self,inputs,box):
#         dp = im2col(inputs,box,box) #change to vars
#         dp = reshape_x(dp)
        self.forward(inputs)
        return self.guess
    

In [371]:
def im2col(matrix,box_width,box_height,stride=1): #need to fix stride
    height = len(matrix)
    width = len(matrix.T)

    ret_height = box_width*box_height
    ret_width = int(((width-box_width)/stride+1)**2)
    
    im_matrix = np.zeros((1,box_width*box_height))
    for i in range(height):
        for j in range(width):
            if (j+box_width <= width) and (i+box_height <= height):
                im_matrix = np.append(im_matrix,
                                      matrix[i:i+box_height,j:j+box_width].reshape(box_width*box_height))
            else:
                break
    val_matrix = im_matrix.T[box_width*box_height:]
    ret_matrix = val_matrix.reshape((ret_height,ret_width))
    return ret_matrix
    

In [ ]:
y = f1['train_labels']
X = np.swapaxes(f1['train_images'],1,0).T
box = 5
h = np.array([im2col(X[i],box,box) for i in range(len(X))]) #change to vars
X = reshape_x(X)
X,y = sklearn.utils.shuffle(X,y,random_state=0)

In [ ]:
meanNeural = NeuralNet(10,100,box,20)
meanNeural.train(X,y,100000,box)

In [ ]:
var = 50123
plt.imshow(X[var])

In [ ]:
print(meanNeural.predict(X[var],box))